In [46]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Charger les données depuis un fichier CSV
data = pd.read_csv(r"C:\Users\33658\PycharmProjects\Act-O-Matic\images_acteurs\acteurs.csv")

# Charger et prétraiter les images
def preprocess_data(data):
    images, ages = [], []
    for i, row in data.iterrows():
        img = cv2.imread(row["image_path"])  # Charger l'image
        img = cv2.resize(img, (224, 224))   # Redimensionner
        img = img / 255.0                   # Normaliser
        images.append(img)
        ages.append(row["age"])
    return np.array(images), np.array(ages)

# Prétraiter les données
images, ages = preprocess_data(data)

# Diviser en ensembles d'entraînement et de validation
x_train, x_val, y_train, y_val = train_test_split(images, ages, test_size=0.2, random_state=42)

# Convertir en tenseurs
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(1000)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

# Charger le modèle pré-entraîné MobileNetV2
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='linear')(x)  # Activation linéaire pour une régression

# Construire le modèle
model = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mean_absolute_error",  # Erreur absolue moyenne pour la régression
    metrics=["mae"]  # Suivi de la MAE
)

# # Afficher le résumé du modèle
# model.summary()

# Entraîner le modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100  # Ajustez selon vos besoins
)

# Sauvegarder le modèle
model.save("age_estimation_model.h5")
print("Modèle sauvegardé sous 'age_estimation_model.h5'")


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 38.9921 - mae: 38.9921 - val_loss: 35.8801 - val_mae: 35.8801
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step - loss: 36.3636 - mae: 36.3636 - val_loss: 33.1301 - val_mae: 33.1301
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - loss: 33.2964 - mae: 33.2964 - val_loss: 30.3937 - val_mae: 30.3937
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - loss: 30.6255 - mae: 30.6255 - val_loss: 27.5594 - val_mae: 27.5594
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - loss: 27.6287 - mae: 27.6287 - val_loss: 24.6442 - val_mae: 24.6442
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step - loss: 25.3393 - mae: 25.3393 - val_loss: 21.6344 - val_mae: 21.6344
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - loss: 22.9364 - mae: 22.9364 - val_loss: 18.9915 - val_mae: 18.9915
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - loss: 19.8809 - mae: 19.8809 - val_loss: 16.6080 - val_mae: 16.6080
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━

Modèle sauvegardé sous 'age_estimation_model.h5'


In [49]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
model = load_model("age_estimation_model.h5")

# Fonction pour tester avec une image
def predict_age(image_path):
    # Charger l'image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Erreur : Impossible de charger l'image {image_path}")
        return
    
    # Prétraiter l'image (redimensionner et normaliser)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalisation
    img = np.expand_dims(img, axis=0)  # Ajouter une dimension pour le batch
    
    # Effectuer la prédiction
    predicted_age = model.predict(img)[0][0]  # Prédiction (régression)
    
    # Afficher le résultat
    print(f"Âge estimé : {predicted_age:.2f} ans")

# Tester avec une image
predict_age(r"C:\Users\33658\Downloads\cruise50.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 643ms/step
Âge estimé : 32.63 ans
